<center>Name : Prathvi N Kanchan<br>
UTAID : 1001769152<br>
Subject : INSY 5378<br>
Individual Project #2<br>

## Project Report - Titanic-Machine Learning from Disaster

## 1.Problem statement
Join kaggel at https://www.kaggle.com/ and  compete in the Titanic Project (accessible at https://www.kaggle.com/c/titanic ). Use train.csv ﬁle and predict unseen data on test.csv.
Survived is your target (dependent variable). Following are the features present:<br>
survival ->	Survival	    : 0 = No, 1 = Yes
pclass	 -> Ticket class	: 1 = 1st, 2 = 2nd, 3 = 3rd
sex	     -> Sex  	
Age	     -> Age in years	
sibsp	 -> # of siblings / spouses aboard the Titanic	
parch	 -> # of parents / children aboard the Titanic	
ticket	 -> Ticket number	
fare	 -> Passenger fare	
cabin	 -> Cabin number	
embarked ->	Port of Embarkation	:C = Cherbourg, Q = Queenstown, S = Southampton

## 2. Preprocessing the data
The first step is to clean the data and preprocess it so that we can develop a better model for the prediction. 

In [1]:
import pandas as pd
import numpy as np

# loading the train and test data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# concatinating the 2 datasets so as to clean and preprocess them together
dataset =  pd.concat(objs=[train, test], axis=0).reset_index(drop=True)

# I am planning to drop the PassengerID column in the future. 
#So keeping a copy of it so as to use it in the submission file
PassengerID = test['PassengerId']
dataset.head()

C:\Users\kanch\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0.0,A/5 21171
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1.0,PC 17599
2,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,1.0,STON/O2. 3101282
3,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,1,1.0,113803
4,35.0,NaN,S,8.0500,"Allen, Mr. William Henry",0,5,3,male,0,0.0,373450


In [2]:
# mapping numeric values to sex so as to use it as a dummy variables in future
dataset['Sex']= dataset['Sex'].map({'female':0,'male':1})
dataset.head(1)

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
0,22.0,NaN,S,7.25,"Braund, Mr. Owen Harris",0,1,3,1,1,0.0,A/5 21171


In [3]:
# By using dataset.isnull().sum()/len(dataset), it is evident that Age,EmbarkedFare,Cabin,Survived
# has null values. I am replacing the age by mean of the dataset instead of dropping those rows 
# so as to retain the other useful information. After filling the null values woth mean age,
# I categorized the age in to Baby 0- 1 years, Child 2-16 years, Adult 17-64, old 65-99 years old
mean = dataset["Age"].mean()
dataset['Age']=dataset['Age'].fillna(round(mean))
age_category = pd.cut(dataset.Age,bins=[0,2,17,65,100],labels=['Baby','Child','Adult','Old'])
dataset.insert(6,'Age Category',age_category)
dataset.head(2)

,Age,Cabin,Embarked,Fare,Name,Parch,Age Category,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,Adult,1,3,1,1,0.0,A/5 21171
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,Adult,2,1,0,1,1.0,PC 17599


In [4]:
#making the age category feature to dummy variables
dataset = pd.get_dummies(dataset,columns=['Age Category'])
dataset.head(2)

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,Age Category_Baby,Age Category_Child,Age Category_Adult,Age Category_Old
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,1,1,0.0,A/5 21171,0,0,1,0
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,0,1,1.0,PC 17599,0,0,1,0


In [6]:
#used the mode function to see the highest repeated value in Embarked i.e to see 
#where most people embarked at. Replace the null values in Embarked with that value i.e S

embarked = dataset['Embarked'].mode()
dataset['Embarked'].fillna(embarked)
dataset.head(2)

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,Age Category_Baby,Age Category_Child,Age Category_Adult,Age Category_Old
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,1,1,0.0,A/5 21171,0,0,1,0
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,0,1,1.0,PC 17599,0,0,1,0


In [7]:
#since Embarked has 3 values, using Dummy variables to use that information
dataset = pd.get_dummies(dataset,columns=['Embarked'])
dataset.head(2)

,Age,Cabin,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,Age Category_Baby,Age Category_Child,Age Category_Adult,Age Category_Old,Embarked_C,Embarked_Q,Embarked_S
0,22.0,NaN,7.2500,"Braund, Mr. Owen Harris",0,1,3,1,1,0.0,A/5 21171,0,0,1,0,0,0,1
1,38.0,C85,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,0,1,1.0,PC 17599,0,0,1,0,1,0,0


In [8]:
# using the data in SibSp and Parch to determine if the person travelled alone or had a companion
#creating new columns travelled_alone and companions to save that information
#its possible that a person travelling alone might had less chance of survival and 
# also person travelling with too many people had less chance of survival

dataset['companions'] = dataset['SibSp'] + dataset['Parch']
dataset.loc[dataset['companions'] > 0, 'travelled_alone'] = 'No'
dataset.loc[dataset['companions'] == 0, 'travelled_alone'] = 'Yes'
travel_map={'No':0,'Yes':1}
dataset['travelled_alone'] = dataset['travelled_alone'].map(travel_map)
dataset.head(2)

,Age,Cabin,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,Age Category_Baby,Age Category_Child,Age Category_Adult,Age Category_Old,Embarked_C,Embarked_Q,Embarked_S,companions,travelled_alone
0,22.0,NaN,7.2500,"Braund, Mr. Owen Harris",0,1,3,1,1,0.0,A/5 21171,0,0,1,0,0,0,1,1,0
1,38.0,C85,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,0,1,1.0,PC 17599,0,0,1,0,1,0,0,1,0


In [9]:
# previously we saw that fare was null in test data. So filling it up with mean fare
#its possible that people who paid more fare were in a better cabin and hadhigher chance of survival
#hence retaining that data
mean = dataset["Fare"].mean()
dataset["Fare"]= dataset["Fare"].fillna(mean)
dataset.head(2)

,Age,Cabin,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,Age Category_Baby,Age Category_Child,Age Category_Adult,Age Category_Old,Embarked_C,Embarked_Q,Embarked_S,companions,travelled_alone
0,22.0,NaN,7.2500,"Braund, Mr. Owen Harris",0,1,3,1,1,0.0,A/5 21171,0,0,1,0,0,0,1,1,0
1,38.0,C85,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,0,1,1.0,PC 17599,0,0,1,0,1,0,0,1,0


In [10]:
# categorizing the fare so as to use it as dummy variables next
# the assumption is that people with less fare i.e who belong to 0 category were given less priority
# while rescuing.
dataset.Fare.fillna(dataset.Fare.mean(),inplace=True)
dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
dataset.loc[(dataset['Fare'] > 31) & (dataset['Fare'] <= 99), 'Fare']   = 3
dataset.loc[(dataset['Fare'] > 99) & (dataset['Fare'] <= 250), 'Fare']   = 4
dataset.loc[ dataset['Fare'] > 250, 'Fare'] = 5
dataset['Fare'] = dataset['Fare'].astype(int)
dataset.head(2)

,Age,Cabin,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,Age Category_Baby,Age Category_Child,Age Category_Adult,Age Category_Old,Embarked_C,Embarked_Q,Embarked_S,companions,travelled_alone
0,22.0,NaN,0,"Braund, Mr. Owen Harris",0,1,3,1,1,0.0,A/5 21171,0,0,1,0,0,0,1,1,0
1,38.0,C85,3,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,0,1,1.0,PC 17599,0,0,1,0,1,0,0,1,0


In [11]:
# creating dummy variables for Fare
dataset = pd.get_dummies(dataset,columns=['Fare'])
dataset.head(2)

,Age,Cabin,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,...,Embarked_Q,Embarked_S,companions,travelled_alone,Fare_0,Fare_1,Fare_2,Fare_3,Fare_4,Fare_5
0,22.0,NaN,"Braund, Mr. Owen Harris",0,1,3,1,1,0.0,A/5 21171,...,0,1,1,0,1,0,0,0,0,0
1,38.0,C85,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,0,1,1.0,PC 17599,...,0,0,1,0,0,0,0,1,0,0


In [12]:
# with the Name we can extract the salutation and find out who were the important people travelling. 
#Using people from higher social status are given forst preference in rescue operations. So I have
# classfied the salutations in to 5 ranks and created a new column Salutation to store that value 
#and later create dummy variables for it
# I have used regular expression to extract the salutation from the name

dataset['Salutation'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
dataset['Salutation'] = dataset['Salutation'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr',\
                                            'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], "Imp")
dataset['Salutation'] = dataset['Salutation'].replace('Mlle', "Miss")
dataset['Salutation'] = dataset['Salutation'].replace('Ms', "Miss")
dataset['Salutation'] = dataset['Salutation'].replace('Miss', "Miss")
dataset['Salutation'] = dataset['Salutation'].replace('Mme', "Miss")

salutation = {"Mr": 5, "Miss": 3, "Mrs": 4, "Master": 2, "Imp": 1}
dataset['Salutation'] = dataset['Salutation'].map(salutation)

dataset = pd.get_dummies(dataset,columns=['Salutation'])
dataset.head(2)

,Age,Cabin,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,...,Fare_1,Fare_2,Fare_3,Fare_4,Fare_5,Salutation_1,Salutation_2,Salutation_3,Salutation_4,Salutation_5
0,22.0,NaN,"Braund, Mr. Owen Harris",0,1,3,1,1,0.0,A/5 21171,...,0,0,0,0,0,0,0,0,0,1
1,38.0,C85,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,0,1,1.0,PC 17599,...,0,0,1,0,0,0,0,0,1,0


In [13]:
# we see that pclass has 3 values and we create dummy variables for that
dataset = pd.get_dummies(dataset,columns=['Pclass'])
dataset.head(2)

,Age,Cabin,Name,Parch,PassengerId,Sex,SibSp,Survived,Ticket,Age Category_Baby,...,Fare_4,Fare_5,Salutation_1,Salutation_2,Salutation_3,Salutation_4,Salutation_5,Pclass_1,Pclass_2,Pclass_3
0,22.0,NaN,"Braund, Mr. Owen Harris",0,1,1,1,0.0,A/5 21171,0,...,0,0,0,0,0,0,1,0,0,1
1,38.0,C85,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,0,1,1.0,PC 17599,0,...,0,0,0,0,0,1,0,1,0,0


In [14]:
# dropping the columns which I dont need as we have used that information in making new columns.
# I have dropped Ticket because the information present in it is representative of that present in Fare 
# and Pclass. PassengerID is also dropped as we dont need it to train the model. Cabin is dropped as it
# has many null values and filling it is a tedious task
dataset = dataset.drop(['Name','Cabin','Age','PassengerId','Ticket','SibSp','Parch'], axis=1)
dataset.head(1)

,Sex,Survived,Age Category_Baby,Age Category_Child,Age Category_Adult,Age Category_Old,Embarked_C,Embarked_Q,Embarked_S,companions,...,Fare_4,Fare_5,Salutation_1,Salutation_2,Salutation_3,Salutation_4,Salutation_5,Pclass_1,Pclass_2,Pclass_3
0,1,0.0,0,0,1,0,0,0,1,1,...,0,0,0,0,0,0,1,0,0,1


In [15]:
# I am rearranging the survived column here and pushing it to the left so that I can use the dataframe
# smoothly for defining the X and Y values for the prediction models
dataset = dataset[['Survived'] + [colmn for colmn in dataset.columns if colmn != 'Survived']]
dataset.head(2)

,Survived,Sex,Age Category_Baby,Age Category_Child,Age Category_Adult,Age Category_Old,Embarked_C,Embarked_Q,Embarked_S,companions,...,Fare_4,Fare_5,Salutation_1,Salutation_2,Salutation_3,Salutation_4,Salutation_5,Pclass_1,Pclass_2,Pclass_3
0,0.0,1,0,0,1,0,0,0,1,1,...,0,0,0,0,0,0,1,0,0,1
1,1.0,0,0,0,1,0,1,0,0,1,...,0,0,0,0,0,1,0,1,0,0


In [97]:
# I am defining the train dataset which we will use for training our models
# since dataset is concatenation of train and test data, I can pull the values of train by reading
# till the length of train data. Y_train has survived as it is what we have to predict and is convertedto int
# as it has 0.0 and 1.0 values
# the independent values are all columns other than survived present in the dataset
train = dataset[:len(train)]
Y_train = train["Survived"].astype(int)
X_train = train.drop(labels = ["Survived"],axis = 1)
train.head()

,Survived,Sex,Age Category_Baby,Age Category_Child,Age Category_Adult,Age Category_Old,Embarked_C,Embarked_Q,Embarked_S,companions,...,Fare_4,Fare_5,Salutation_1,Salutation_2,Salutation_3,Salutation_4,Salutation_5,Pclass_1,Pclass_2,Pclass_3
0,0.0,1,0,0,1,0,0,0,1,1,...,0,0,0,0,0,0,1,0,0,1
1,1.0,0,0,0,1,0,1,0,0,1,...,0,0,0,0,0,1,0,1,0,0
2,1.0,0,0,0,1,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,1
3,1.0,0,0,0,1,0,0,0,1,1,...,0,0,0,0,0,1,0,1,0,0
4,0.0,1,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,1


In [98]:
# the test dataset is used to test the model and we drop the Survived column from the dataset here 
# because we have to predict them
test = dataset[len(train):]
X_test = test.drop(labels = ["Survived"],axis = 1)
X_test.head()

,Sex,Age Category_Baby,Age Category_Child,Age Category_Adult,Age Category_Old,Embarked_C,Embarked_Q,Embarked_S,companions,travelled_alone,...,Fare_4,Fare_5,Salutation_1,Salutation_2,Salutation_3,Salutation_4,Salutation_5,Pclass_1,Pclass_2,Pclass_3
891,1,0,0,1,0,0,1,0,0,1,...,0,0,0,0,0,0,1,0,0,1
892,0,0,0,1,0,0,0,1,1,0,...,0,0,0,0,0,1,0,0,0,1
893,1,0,0,1,0,0,1,0,0,1,...,0,0,0,0,0,0,1,0,1,0
894,1,0,0,1,0,0,0,1,0,1,...,0,0,0,0,0,0,1,0,0,1
895,0,0,0,1,0,0,0,1,2,0,...,0,0,0,0,0,1,0,0,0,1


## 3. Models Used For Prediction
I have used Logistic Regression, Support Vector Machines, KNN, Random Forest, Decision Tree, Gaussian Naive Bayes algorithms for the prediction of Survived.

In [99]:
#using the logistic regression model. Here I have fit the model with the train data 
# and used cross validation and have used accuracy as the scoring value. Have used cv= 10 as it is the 
# best cv value
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
logreg_model = LogisticRegression(solver='liblinear')
accuracy_logreg = cross_val_score(logreg_model, X_train, Y_train, cv=10, scoring='accuracy')
logreg_model.fit(X_train,Y_train)
print("Mean Accuracy Score for Log Reg: {}".format(np.mean(accuracy_logreg)))

Mean Accuracy Score for Log Reg: 0.8261014640789922


In [100]:
#using the test data set and predicting survived . The result is writting to csv file which is uploaded to
# the kaggle website to check our prediction accuracy
#passengerId is used to map the survived value
y_test = logreg_model.predict(X_test)
output = pd.DataFrame({"PassengerId":PassengerID,'Survived':y_test})
output.to_csv('submission.csv',index= False)
k_accuracy_logreg = 0.78468
# Got 0.78468 score in Kaggle

In [85]:
# training the Support Vector Machines algorithm. Using crossvalidation=10 and then predicting the 
# accuracy of the model
from sklearn.svm import SVC
svc = SVC(gamma='auto',kernel='linear')
accuracy_svc = cross_val_score(svc, X_train, Y_train, cv=10, scoring='accuracy')
svc.fit(X_train,Y_train)
print("Mean Accuracy Score for SVM : {}".format(np.mean(accuracy_svc)))

Mean Accuracy Score for SVM : 0.835015038020656


In [86]:
#testing the model with test data and saving the result in csv file to check in Kaggle
y_test = svc.predict(X_test)
output = pd.DataFrame({"PassengerId":PassengerID,'Survived':y_test})
output.to_csv('submission.csv',index= False)
k_accuracy_svc = 0.78947
#0.78947 score in Kaggle website

In [87]:
# KNN or k-Nearest Neighbors model is trained
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
accuracy_knn = cross_val_score(knn, X_train, Y_train, cv=10, scoring='accuracy')
knn.fit(X_train,Y_train)
print("Mean Accuracy Score for KNN : {}".format(np.mean(accuracy_knn)))

Mean Accuracy Score for KNN : 0.8182731812507094


In [88]:
#using test data to predict
y_test = knn.predict(X_test)
output = pd.DataFrame({"PassengerId":PassengerID,'Survived':y_test})
output.to_csv('submission.csv',index= False)
k_accuracy_knn = 0.78468
#0.78468 accuracy score in kaggles website

In [89]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

randomforest = RandomForestClassifier(n_estimators=25, random_state=0)
accuracy_rf = cross_val_score(randomforest, X_train, Y_train, cv=10, scoring='accuracy')
randomforest.fit(X_train, Y_train)
print("Mean Accuracy Score for Random Forest: {}".format(np.mean(accuracy_rf)))

Mean Accuracy Score for Random Forest: 0.8215940301895358


In [90]:
X_test = test.drop(labels = ["Survived"],axis = 1)
Y_test = randomforest.predict(X_test)
output = pd.DataFrame({"PassengerId":PassengerID,'Survived':Y_test})
output.to_csv('submission.csv',index= False)
k_accuracy_rf = 0.7703
#0.7703 accuracy score in Kaggle website 

In [91]:
# using gridsearchcv to find the best parameter to apply to the decisiontree
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
param_grid = {'max_depth': [2,3,4,5,6,7,8]}
cv = StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
grid = GridSearchCV(model, param_grid, cv = cv, scoring='accuracy',
                    return_train_score=True)
grid.fit(X,y)
print("Best Parameter: {}".format(grid.best_params_))

Best Parameter: {'max_depth': 4}


In [92]:
#fitting the decision tree with the best parameter
decisiontree = DecisionTreeClassifier(max_depth=4)
accuracy_dt = cross_val_score(decisiontree, X_train, Y_train, cv=10, scoring='accuracy')
decisiontree.fit( X_train, Y_train)
print("Mean Accuracy Score for decision tree: {}".format(np.mean(accuracy_dt)))

Mean Accuracy Score for decision tree: 0.8203955283168767


In [93]:
#using the testdata and predicting using deciosn tree model
y_test = decisiontree.predict(X_test)
output = pd.DataFrame({"PassengerId":PassengerID,'Survived':y_test})
output.to_csv('submission.csv',index= False)
k_accuracy_dt=0.7894
# 0.7894 accuracy score in kaggle website

In [94]:
# Gaussian Naive Bayes model is trained using the train data. 
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

gaussian = GaussianNB()
accuracy_gnb = cross_val_score(gaussian, X_train, Y_train, cv=10, scoring='accuracy')
gaussian.fit(X_train, Y_train)
print("Mean Accuracy Score for gaussian naive Bayes: {}".format(np.mean(accuracy_gnb)))

Mean Accuracy Score for gaussian naive Bayes: 0.7655629326977642


In [95]:
#predicting using test data
y_test = gaussian.predict(X_test)
output = pd.DataFrame({"PassengerId":PassengerID,'Survived':y_test})
output.to_csv('submission.csv',index= False)
k_accuracy_gnb = 0.74
#0.74 accuracy score in Kaggle website

## Conclusion
Below is the accuracy score for the models used above in descending order of accuracy.

In [101]:
# Comparing accuracies of each models
models_used = pd.DataFrame({
    'Model': ['Logistic Regression','Support Vector Machine','KNN','Random Forest','Decision Tree','Gaussian Naive Bayes'],
    'Accuracy Score': [np.mean(accuracy_logreg), np.mean(accuracy_svc), np.mean(accuracy_knn),np.mean(accuracy_rf), np.mean(accuracy_dt), np.mean(accuracy_gnb)],
    'Kaggle Accuracy Score': [k_accuracy_logreg, k_accuracy_svc, k_accuracy_knn,k_accuracy_rf, k_accuracy_dt, k_accuracy_gnb]})
models_used.sort_values(by='Kaggle Accuracy Score', ascending=False)

,Model,Accuracy Score,Kaggle Accuracy Score
1,Support Vector Machine,0.835015,0.78947
4,Decision Tree,0.820396,0.78940
0,Logistic Regression,0.826101,0.78468
2,KNN,0.818273,0.78468
3,Random Forest,0.821594,0.77030
5,Gaussian Naive Bayes,0.765563,0.74000


As per the above comparision, for the given dataset, Support Vector Machine is better compared to the other models discussed here as the Kaggle accuracy score for predicting the survived variable in test dataset is more compared to others for the independent variables we considered. 